## Load state

In [231]:
import sys
from pathlib import Path

repo_dir = Path('..').resolve()
assert repo_dir.name == 'hicetnunc-dataset', repo_dir
if str(repo_dir) not in sys.path:
    sys.path.append(str(repo_dir))

import src.reload; src.reload.reload()

nft_state_log = src.utils.read_json(src.config.nft_state_log_file)
nft_state = src.contracts.nft_state.NFTState()
nft_state_replayer = src.contracts.state_utils.StateReplayer(nft_state_log, nft_state)

ah_state_log = src.utils.read_json(src.config.ah_state_log_file)
ah_state = src.contracts.art_house_state.ArtHouseState()
ah_state_replayer = src.contracts.state_utils.StateReplayer(ah_state_log, ah_state)

## Filter money trs

In [232]:
money_trs = []

for tr in src.iter_tr.iter_tr():
    tr_volume = 0
    for op in tr['ops']:
        if op['status'] != 'applied':
            continue
        assert op['volume'] >= 0
        tr_volume += op['volume']
    if tr_volume > 0:
        money_trs.append(tr)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:06<00:00,  3.48it/s]

## Classify money transactions

In [306]:
from collections import defaultdict


class MoneyState:
    def __init__(self):
        self.reset()

    def reset(self):
        self.balances = Counter()
        self.addrs = defaultdict(lambda: {
            'total_bought': 0,
            'total_sold': 0,
            'events': [],
        })
        self.tokens = defaultdict(lambda: {
            'events': [],
        })
        self.comission_wallet = {
            'comission_income': 0,
            'percent_income': 0,
            'other_income': 0,
            'spending': 0,
            'events': [],
        }

    def apply_swap_sell(self, row_id, token_id, token_count, payer, beneficiary, price, comission):
        event = {
            'row_id': row_id,
            'type': 'swap_sell',
            'token_id': token_id,
            'token_count': token_count,
            'payer': payer,
            'beneficiary': beneficiary,
            'price': price,
            'comission': comission,
        }
        self.addrs[payer]['total_bought'] += price
        self.addrs[beneficiary]['total_sold'] += price - comission
        self.addrs[payer]['events'].append(event)
        self.addrs[beneficiary]['events'].append(event)
        self.tokens[token_id]['events'].append(event)
        self.comission_wallet['comission_income'] += comission
        self.comission_wallet['events'].append(event)

        self.balances[payer] -= price
        self.balances[beneficiary] += price - comission
        self.balances[src.config.name2addr['comission_wallet']] += comission

    def apply_comission_wallet_baking_percent(self, row_id, sender, volume):
        assert sender == src.config.name2addr['baking_benjamins']
        assert volume > 0
        event = {
            'row_id': row_id,
            'type': 'comission_wallet_baking_percent',
            'sender': sender,
            'receiver': src.config.name2addr['comission_wallet'],
            'volume': volume,
        }
        self.comission_wallet['events'].append(event)
        self.comission_wallet['percent_income'] += volume
        self.balances[src.config.name2addr['baking_benjamins']] -= volume
        self.balances[src.config.name2addr['comission_wallet']] += volume

    def apply_comission_wallet_spending(self, row_id, receiver, volume):
        assert volume > 0
        event = {
            'row_id': row_id,
            'type': 'comission_wallet_spending',
            'sender': src.config.name2addr['comission_wallet'],
            'receiver': sender,
            'volume': volume,
        }
        self.comission_wallet['events'].append(event)
        self.comission_wallet['spending'] += volume    
        self.balances[receiver] += volume
        self.balances[src.config.name2addr['comission_wallet']] -= volume


from collections import Counter

money_state = MoneyState()
money_state = src.contracts.state_utils.StateRecorder(money_state)

for tr in money_trs:

    nft_calls = Counter()
    last_nft_op = None
    for op in tr['ops']:
        if op['type'] == 'transaction' and op['receiver'] == src.config.name2addr['nft_contract']:
            nft_calls[op['parameters']['call']] += 1
            last_nft_op = op
    nft_calls = dict(nft_calls)

    money_delta = get_tr_money_delta(tr)
    known_money_delta = {
        src.config.addr2name[addr]: delta
        for addr, delta in money_delta.items()
        if addr in src.config.addr2name
    }

    if nft_calls == {'transfer': 1}:
        assert last_nft_op['parameters']['call'] == 'transfer'
        transfer = last_nft_op['parameters']['value']['transfer']
        assert len(transfer) == 1
        sender = transfer[0]['from_']

        # when the first transfer between users (with non-zero price) will happen,
        # this assert will fail
        assert sender == src.config.name2addr['art_house_contract']
        assert money_delta[src.config.name2addr['art_house_contract']] < 1e-10

        assert len(transfer[0]['txs']) == 1
        receiver = transfer[0]['txs'][0]['to_']

        token_id = int(transfer[0]['txs'][0]['token_id'])
        token_count = int(transfer[0]['txs'][0]['amount'])
        comission_wallet_income = money_delta[src.config.name2addr['comission_wallet']]

        beneficiaries = {}
        payers = {}
        for addr, delta in money_delta.items():
            if addr in [
                src.config.name2addr['art_house_contract'],
                src.config.name2addr['comission_wallet'],
            ]:
                continue
            if delta > 0:
                beneficiaries[addr] = delta
            elif delta < 0:
                payers[addr] = delta

        if receiver == src.config.name2addr['comission_wallet']:
            assert len(payers) == 0
            assert len(beneficiaries) == 1
            payer = src.config.name2addr['comission_wallet']
            assert comission_wallet_income < 0
            price = abs(comission_wallet_income) / (1.0 - 0.025)
            comission = price * 0.025

        elif len(beneficiaries) == 0:
            assert len(payers) == 1
            payer = list(payers)[0]
            assert money_delta[payer] == -comission_wallet_income
            assert comission_wallet_income > 0
            price = abs(comission_wallet_income)
            beneficiary = src.config.name2addr['comission_wallet']
            comission = price * 0.025

        else:
            assert len(payers) == 1
            assert len(beneficiaries) == 1
            assert comission_wallet_income > 0
            comission = comission_wallet_income
            payer = list(payers)[0]
            price = -money_delta[payer]
            beneficiary = list(beneficiaries)[0]

        assert type(payer) is str
        assert type(beneficiary) is str
        assert price > 0
        assert comission > 0
        assert abs(comission - price * 0.025) < 1e-6

        money_state.apply_swap_sell(
            row_id=last_nft_op['row_id'],
            token_id=token_id,
            token_count=token_count,
            payer=payer,
            beneficiary=beneficiary,
            price=price,
            comission=comission,
        )
        continue

    assert nft_calls == {}
    known_money_addrs = set(known_money_delta.keys())

    assert 'comission_wallet' in known_money_addrs

    if 'baking_benjamins' in known_money_addrs:
        assert known_money_addrs == {'baking_benjamins', 'comission_wallet'}
        assert known_money_delta['baking_benjamins'] < 0
        assert known_money_delta['comission_wallet'] > 0
        assert known_money_delta['comission_wallet'] < -known_money_delta['baking_benjamins']
        money_state.apply_comission_wallet_baking_percent(
            row_id=tr['ops'][0]['row_id'],
            sender=src.config.name2addr['baking_benjamins'],
            volume=known_money_delta['comission_wallet'],
        )
        continue

    assert known_money_addrs == { 'comission_wallet' }
    assert known_money_delta['comission_wallet'] < 0
    other_addrs = set(money_delta.keys()) - {src.config.name2addr['comission_wallet']}
    assert len(other_addrs) == 1
    other_addr = list(other_addrs)[0]
    volume = money_delta[other_addr]
    assert volume > 0
    assert money_delta == {
        src.config.name2addr['comission_wallet']: -volume,
        other_addr: volume
    }
    money_state.apply_comission_wallet_spending(
        row_id=tr['ops'][0]['row_id'],
        receiver=other_addr,
        volume=volume,
    )

In [234]:
comission_wallet_initial_volume = 102.02

In [235]:
src.config.name2addr['comission_wallet']

'tz1UBZUkXpKGhYsP5KtzDNqLLchwF4uHrGjw'

In [300]:
comission_wallet_initial_volume + money_state.balances[src.config.name2addr['comission_wallet']]

1788.859149000093

In [301]:
comission_wallet_initial_volume + money_state.comission_wallet['comission_income'] + money_state.comission_wallet['percent_income'] - money_state.comission_wallet['spending'] - money_state.addrs[src.config.name2addr['comission_wallet']]['total_bought'] + money_state.addrs[src.config.name2addr['comission_wallet']]['total_sold']

1788.8591490001595

## Check comission wallet transactions

In [305]:
i_dianov_addr = 'tz1MwvWz611pLwYdFMZpWCEsh247MEYtm5ii'

In [309]:
money_state.addrs[i_dianov_addr]

{'total_bought': 22.6,
 'total_sold': 147.22499999999985,
 'events': [{'row_id': 42597772,
   'type': 'swap_sell',
   'token_id': 898,
   'token_count': 1,
   'payer': 'tz1MwvWz611pLwYdFMZpWCEsh247MEYtm5ii',
   'beneficiary': 'tz2VN3fWzqHE44F6zedJB7UtALKEh5WhqAJJ',
   'price': 1,
   'comission': 0.025},
  {'row_id': 42605395,
   'type': 'swap_sell',
   'token_id': 1091,
   'token_count': 1,
   'payer': 'tz2VN3fWzqHE44F6zedJB7UtALKEh5WhqAJJ',
   'beneficiary': 'tz1MwvWz611pLwYdFMZpWCEsh247MEYtm5ii',
   'price': 1,
   'comission': 0.025},
  {'row_id': 42606482,
   'type': 'swap_sell',
   'token_id': 1091,
   'token_count': 1,
   'payer': 'tz2SV4pB3uxG7pzqw5rvZAh4agPbuYJnKxsS',
   'beneficiary': 'tz1MwvWz611pLwYdFMZpWCEsh247MEYtm5ii',
   'price': 1,
   'comission': 0.025},
  {'row_id': 42606801,
   'type': 'swap_sell',
   'token_id': 1091,
   'token_count': 1,
   'payer': 'tz1Ro4mBmTSx1crHwQx6B41w2oG8B82WfeGa',
   'beneficiary': 'tz1MwvWz611pLwYdFMZpWCEsh247MEYtm5ii',
   'price': 1,
   'c

In [308]:
money_state.balances[i_dianov_addr]

124.62499999999984

In [249]:
money_state_r = MoneyState()
money_state_replayer = src.contracts.state_utils.StateReplayer(money_state.log, money_state_r)

In [304]:
cw_balance = comission_wallet_initial_volume

first_op = None

for tr in money_trs: # src.iter_tr.iter_tr():
    for op in tr['ops']:
        if op['status'] != 'applied':
            continue
        if op['volume'] == 0:
            continue
        if first_op is None:
            first_op = op
        if op['sender'] == src.config.name2addr['comission_wallet']:
            cw_balance -= op['volume']
        if op['receiver'] == src.config.name2addr['comission_wallet']:
            cw_balance += op['volume']
    # money_state_replayer.replay_to_after_row_id(tr['ops'][-1]['row_id'])
    # print(cw_balance, comission_wallet_initial_volume + money_state_r.balances[src.config.name2addr['comission_wallet']])

cw_balance

1788.8591490001104

In [264]:
src.config.name2addr['comission_wallet']

'tz1UBZUkXpKGhYsP5KtzDNqLLchwF4uHrGjw'

In [266]:
102.043724 - 0.975

101.068724

In [271]:
0.975 == 1.0 * (1.0 - 0.025)

True

In [263]:
get_tr_money_delta(tr)

{'tz1UBZUkXpKGhYsP5KtzDNqLLchwF4uHrGjw': -0.975,
 'KT1Hkg5qeNhfwpKW4fXvq7HGZB9z2EnmCCA9': 0.0,
 'tz1Rux3pGrp2KpZUBKvg3VQ4edRYVJsapr4z': 0.975}

In [252]:
tr['ops'][-1]['row_id']

43940499

In [224]:
first_op['time']

'2021-02-28T14:42:45Z'

In [28]:
def get_tr_money_delta(tr):
    res = {}
    for op in tr['ops']:
        if op['volume'] == 0:
            continue
        sender, receiver = op['sender'], op['receiver']
        res[sender] = res.get(sender, 0) - op['volume']
        res[receiver] = res.get(receiver, 0) + op['volume']
    return res


def get_addr_signature(addr):
    return src.config.addr2name.get(addr, '*')


def get_op_signature(op):
    if op['status'] != 'applied':
        return op['status']
    if op['type'] != 'transaction':
        return op['type']
    call = op.get('parameters', {'call': ''})['call']
    if op['volume'] > 0:
        return (
            call,
            get_addr_signature(op['sender']) + ' > ' + get_addr_signature(op['receiver']),
        )
    return call


def get_tr_signature(tr):
    ext_count = 0
    res = tuple()
    has_any_applied = False
    for op in tr['ops']:
        if op['type'] == 'reveal':
            continue
        if op['status'] == 'applied':
            has_any_applied = True
        if op['sender'] not in src.config.addr2name and op.get('receiver') not in src.config.addr2name:
            ext_count += 1
        else:
            res += (get_op_signature(op),)
    if not has_any_applied:
        return tr['ops'][0]['status']
    res = (ext_count > 0,) + res
    return res


In [5]:
nft_state_replayer.replay_to_end()
ah_state_replayer.replay_to_end()

In [16]:
import requests

r = requests.get('https://api.tzstats.com/explorer/op/op8eBZNRQY8SFXZJmYpHczAdSGhzPjpFBRr3kUxupYegfAPRuzu')
r.text

'[{"row_id":43782809,"hash":"op8eBZNRQY8SFXZJmYpHczAdSGhzPjpFBRr3kUxupYegfAPRuzu","type":"transaction","block":"BKsCu8zwXR3tkM5BcvvJWkENFArGdHfTzXCMfzR3JbNyBLxVmfA","time":"2021-03-22T20:44:38Z","height":1396278,"cycle":340,"counter":11763276,"op_l":3,"op_p":4,"op_c":0,"op_i":0,"status":"applied","is_success":true,"is_contract":true,"gas_limit":93655,"gas_used":68475,"gas_price":0.14284,"storage_limit":209,"storage_size":1831924,"storage_paid":0,"volume":0,"fee":0.009781,"has_data":true,"days_destroyed":0,"parameters":{"entrypoint":"mint_OBJKT","call":"mint_OBJKT","branch":"RLR","id":4,"value":{"address":"tz1bJwj5KBMc3UFk3ijykA9c1sGDmw4FEy24","amount":"100","metadata":"ipfs://QmaHsLuyrxex9CusGS29i4taHA6AWPvfircvMQbMhFBboy","royalties":"100"}},"storage":{"value":{"curate":"KT1TybhR7XraG75JFYKSrh7KnxukMBT5dor6","genesis":"2021-04-15T02:09:41Z","hdao":"KT1AFA2mwNUMNd4SsujE1YYp29vd8BZejyKW","locked":"true","manager":"tz1UBZUkXpKGhYsP5KtzDNqLLchwF4uHrGjw","metadata":"521","objkt":"KT1RJ6Pbj

In [11]:
nft_state_replayer.replay_to_end()

In [13]:
nft_state.tokens

{152: {'creator': 'tz1UBZUkXpKGhYsP5KtzDNqLLchwF4uHrGjw',
  'mint_count': 1,
  'info_ipfs': 'ipfs://Qma11k6ahPRXGVV7RgHNuLuwB9PqQB2MVJUtNNXJ7dQeAr',
  'mint_row_id': 42212376,
  'own_counts': {'tz1UBZUkXpKGhYsP5KtzDNqLLchwF4uHrGjw': 1,
   'KT1Hkg5qeNhfwpKW4fXvq7HGZB9z2EnmCCA9': 0},
  'transfers': [{'from': 'tz1UBZUkXpKGhYsP5KtzDNqLLchwF4uHrGjw',
    'to': 'KT1Hkg5qeNhfwpKW4fXvq7HGZB9z2EnmCCA9',
    'count': 1,
    'row_id': 42467796},
   {'from': 'KT1Hkg5qeNhfwpKW4fXvq7HGZB9z2EnmCCA9',
    'to': 'tz1UBZUkXpKGhYsP5KtzDNqLLchwF4uHrGjw',
    'count': 1,
    'row_id': 42468333}]},
 153: {'creator': 'tz1UBZUkXpKGhYsP5KtzDNqLLchwF4uHrGjw',
  'mint_count': 2,
  'info_ipfs': 'ipfs://QmeaqRBUiw4cJiNKEcW2noc7egLd5GgBqLcHHqUhauJAHN',
  'mint_row_id': 42220283,
  'own_counts': {'tz1UBZUkXpKGhYsP5KtzDNqLLchwF4uHrGjw': 2},
  'transfers': []},
 154: {'creator': 'tz1UBZUkXpKGhYsP5KtzDNqLLchwF4uHrGjw',
  'mint_count': 2,
  'info_ipfs': 'ipfs://QmUxvVm8vxZ8zzxD8zU4ZCsKUPHc888ztrkaFcGasEKzbp',
  'mint_ro

In [72]:
import requests

req = requests.get(f'https://api.tzkt.io/v1/accounts/tz1RnsQi6BaYpm6sMMTkVqGk5Jk6BKpmTp8C/metadata')
req.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [67]:
req.status_code

204

In [30]:
import sys
import time
import datetime
from pathlib import Path
from tqdm import tqdm
import requests
from collections import Counter

repo_dir = Path('..').resolve()
assert repo_dir.name == 'hicetnunc-dataset', repo_dir
if str(repo_dir) not in sys.path:
    sys.path.append(str(repo_dir))

import lib.utils
import lib.iter_tr


stamp_step = 100000
config_hash = '4ff66f8cc8'
transactions_dir = repo_dir / 'cache' / 'transactions' / config_hash
assert transactions_dir.is_dir()

name2addr = {
    'art_house_contract': 'KT1Hkg5qeNhfwpKW4fXvq7HGZB9z2EnmCCA9',
    'curate_contract': 'KT1TybhR7XraG75JFYKSrh7KnxukMBT5dor6',
    'hdao_contract': 'KT1AFA2mwNUMNd4SsujE1YYp29vd8BZejyKW',
    'nft_contract': 'KT1RJ6PbjHpwc3M5rw5s2Nbmefwbuwbdxton',
    'comission_wallet': 'tz1UBZUkXpKGhYsP5KtzDNqLLchwF4uHrGjw',
}
addr2name = { addr: name for name, addr in name2addr.items() }

In [7]:
money_trs = []

for tr in lib.iter_tr.iter_tr(transactions_dir, stamp_step):
    tr_addrs = set()
    tr_volume = 0
    for op in tr['ops']:
        tr_volume += op['volume']
        tr_addrs.add(op['sender'])
        if 'receiver' in op:
            tr_addrs.add(op['receiver'])
    if not (tr_addrs & known_addrs) or tr_volume == 0:
        continue
    money_trs.append(tr)

len(money_trs)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:04<00:00,  4.57it/s]

10630

In [80]:
def get_tr_money_delta(tr):
    res = {}
    for op in tr['ops']:
        if op['volume'] == 0:
            continue
        sender, receiver = op['sender'], op['receiver']
        res[sender] = res.get(sender, 0) - op['volume']
        res[receiver] = res.get(receiver, 0) + op['volume']
    return res


def get_addr_signature(addr):
    return addr2name.get(addr, '*')


def get_op_signature(op):
    if op['status'] != 'applied':
        return op['status']
    if op['type'] != 'transaction':
        return op['type']
    call = op.get('parameters', {'call': ''})['call']
    if op['volume'] > 0:
        return (
            call,
            get_addr_signature(op['sender']) + ' > ' + get_addr_signature(op['receiver']),
        )
    return call


def get_tr_signature(tr):
    ext_count = 0
    res = tuple()
    has_any_applied = False
    for op in tr['ops']:
        if op['type'] == 'reveal':
            continue
        if op['status'] == 'applied':
            has_any_applied = True
        if op['sender'] not in addr2name and op.get('receiver') not in addr2name:
            ext_count += 1
        else:
            res += (get_op_signature(op),)
    if not has_any_applied:
        return tr['ops'][0]['status']
    res = (ext_count > 0,) + res
    return res


from collections import Counter

tr_signatures = Counter()

for tr in money_trs:
    tr_signature = get_tr_signature(tr)
    if tr_signature == (False, ('', 'comission_wallet > *')):
        # assert len(tr['ops']) == 1
        # print(tr['hash'])
        # print(tr['time'], tr['ops'][0]['receiver'], tr['ops'][0]['volume'])
        pass
    if tr_signature == (False,
        ('collect', '* > art_house_contract'),
        ('', 'art_house_contract > *'),
        ('', 'art_house_contract > comission_wallet'),
        ('', 'art_house_contract > *'),
        'hDAO_batch',
        'transfer'):
            break
    tr_signatures[tr_signature] += 1

tr_signatures.most_common()

[((True, ('', '* > comission_wallet')), 2),
 ((False,
   ('collect', 'comission_wallet > art_house_contract'),
   ('', 'art_house_contract > *'),
   ('', 'art_house_contract > comission_wallet'),
   ('', 'art_house_contract > *'),
   'hDAO_batch',
   'transfer'),
  1)]

In [81]:
print(get_tr_money_delta(tr))

{'tz1Rux3pGrp2KpZUBKvg3VQ4edRYVJsapr4z': -10, 'KT1Hkg5qeNhfwpKW4fXvq7HGZB9z2EnmCCA9': 0.0, 'tz1ZdMfzmWLb8mu22jE7rZ8Y3t9iKezM68cq': 9.75, 'tz1UBZUkXpKGhYsP5KtzDNqLLchwF4uHrGjw': 0.25}
